# Introduction

This tutorial demonstrates angle resolved virtual backscatter electron
(BSE) imaging.

For learning purposes, we recommend to use this notebook alongside our
[user guide](https://kikuchipy.org).

The data consists of 29 800 EBSD patterns from recrystallized
polycrystalline Nickel. It is available
[via Zenodo](https://zenodo.org/record/3265037).

This functionality has been checked to run in kikuchipy-0.2.0 (May 2020). Bugs
are always possible, do not trust the code blindly, and if you experience any
issues, report them
[in our issue tracker](https://github.com/kikuchipy/kikuchipy-demos/issues).

Access all methods' documentation (docstrings) when the cursor is within the
function parantheses, like `function(CURSOR HERE)`, by holding `SHIFT` and
pressing `TAB`.

# Contents

1. Loading and data inspection
2. Background correction
3. Interactive plotting  
    i. Imaging  
    ii. Select a data region
4. Multiple images from detector grid tiles
5. RGB images  
    i. From detector grid tiles  
    ii. From Gaussian fitting parameters
6. Processing virtual image intensities
    (normalization/rescaling/contrast stretching)

Import kikuchipy and other necessary libraries

In [1]:
%matplotlib qt5
import hyperspy.api as hs
import matplotlib.pyplot as plt
import numpy as np
import kikuchipy as kp

# 1. Loading and data inspection

In [5]:
data = "/home/hakon/phd/data/ni/2020/1/nordif/Pattern.dat"
s = kp.load(data, lazy=False)  # lazy=True to not load into memory before calling s.compute()

Inspect the metadata

In [ ]:
s.metadata

Inspect the signal (detector) and navigation (sample) axes

In [8]:
s.axes_manager

<Axes manager, axes: (200, 149|60, 60)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               x |    200 |      0 |       0 |     1.5 |     um 
               y |    149 |      0 |       0 |     1.5 |     um 
---------------- | ------ | ------ | ------- | ------- | ------ 
              dx |     60 |        |       0 |       1 |     um 
              dy |     60 |        |       0 |       1 |     um

Plot the patterns

In [ ]:
s.plot()

# 2. Background correction

Remove the static background, but keeping retain the relative intensities. This
correction retains the variation in the the backscatter electron yield from
sample point to sample point

In [37]:
s.remove_static_background(relative=True)

Removing the static background:
[########################################] | 100% Completed |  2.5s


# 3. Interactive plotting

## 3.1 Imaging

Plot an interactive virtual image by integrating the intensities within a rectangular subset of pixels on the detector

In [38]:
roi = hs.roi.RectangularROI(left=0, top=0, right=10, bottom=10)

In [39]:
s.plot_virtual_bse_intensity(roi)

Get the virtual BSE image associated with the last integration window used interactively

In [9]:
vbse = s.get_virtual_bse_intensity(roi)

In [10]:
vbse

<VirtualBSEImage, title: Virtual backscatter electron image, dimensions: (|200, 149)>

Plot the virtual BSE image

In [11]:
vbse.plot()

Inspect the metadata

In [12]:
vbse.metadata

├── Acquisition_instrument
│   └── SEM
│       ├── Detector
│       │   └── EBSD
│       │       ├── azimuth_angle = 0.0
│       │       ├── binning = -1.0
│       │       ├── detector = NORDIF UF1100
│       │       ├── elevation_angle = 0.0
│       │       ├── exposure_time = 0.0035
│       │       ├── frame_number = -1
│       │       ├── frame_rate = 202
│       │       ├── gain = 0.0
│       │       ├── grid_type = square
│       │       ├── manufacturer = NORDIF
│       │       ├── sample_tilt = 70.0
│       │       ├── scan_time = 148
│       │       ├── static_background = array([[84, 87, 90, ..., 27, 29, 30],
       [87, 90, 93, ..., 27, 28, 30],
   ...  80, 82, ..., 28, 26, 26],
       [76, 78, 80, ..., 26, 26, 25]], dtype=uint8)
│       │       ├── version = 3.1.2
│       │       ├── xpc = -1.0
│       │       ├── ypc = -1.0
│       │       └── zpc = -1.0
│       ├── beam_energy = 20.0
│       ├── magnification = 200
│       ├── microscope = Hitachi SU-6600
│       └── worki

Save the virtual BSE image as a 32-bit TIFF file

In [14]:
vbse.change_dtype(np.float32)

In [ ]:
vbse.save("vbse1.tif")

## 3.2 Select a data region

Plot the data with an adjustable marker indicating where to crop the data region

In [40]:
reg = hs.roi.RectangularROI(left=0, top=0, right=10, bottom=10)
s.plot()
reg.add_widget(s)

Crop the data set based on the region defined above

In [41]:
s2 = reg(s)

Calculate the mean diffraction pattern from the selected region

In [42]:
s2_mean = s2.mean(axis=(0, 1))

Plot the mean diffraction pattern from the selected region

In [43]:
s2_mean.plot()

# 4. Multiple images from detector grid tiles

Initialize a VirtualBSEImageGenerator

In [44]:
vbse_gen = kp.generators.VirtualBSEGenerator(s)

In [45]:
vbse_gen

VirtualBSEGenerator for <EBSD, title: Pattern, dimensions: (200, 149|60, 60)>

Inspect the generator detector grid

In [46]:
vbse_gen.grid_shape

(5, 5)

In [47]:
vbse_gen.plot_grid()

<EBSD, title: Pattern, dimensions: (|60, 60)>

Change the grid shape

In [48]:
vbse_gen.grid_shape = (6, 6)

Compute one virtual BSE image per detector grid tile

In [49]:
vbse_grid_images = vbse_gen.get_images_from_grid()

In [50]:
vbse_grid_images

<VirtualBSEImage, title: , dimensions: (6, 6|200, 149)>

Plot the virtual BSE images

In [51]:
vbse_grid_images.plot()

Save the BSE images as a 32-bit TIFF stack

In [24]:
vbse_grid_images.unfold_navigation_space()

True

In [ ]:
vbse_grid_images

In [26]:
vbse_grid_images.save("vbse2.tif")

# 5. RGB images

## 5.1 From detector grid tiles

## 5.2 From Gaussian fitting parameters

# 6. Processing virtual image intensities